In [2]:
import pandas as pd

# Question 01

In [1]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet

--2023-05-15 19:18:26--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.244.96.180, 18.244.96.218, 18.244.96.25, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.244.96.180|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38139949 (36M) [application/x-www-form-urlencoded]
Saving to: ‘yellow_tripdata_2022-01.parquet’

yellow_tripdata_202 100%[===================>]  36.37M  2.25MB/s    in 18s     

2023-05-15 19:18:44 (2.02 MB/s) - ‘yellow_tripdata_2022-01.parquet’ saved [38139949/38139949]

--2023-05-15 19:18:45--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.244.96.180, 18.244.96.218, 18.244.96.25, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.244.96.180|:443... connected

In [3]:
df_jan = pd.read_parquet('yellow_tripdata_2022-01.parquet')
df_jan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


In [4]:
print(f'Answer: {df_jan.shape[1]} columns')

Answer: 19 columns


## Question 2

In [11]:
df_jan['duration'] = (df_jan.tpep_dropoff_datetime - df_jan.tpep_pickup_datetime).dt.total_seconds() / 60
df_jan.duration

0          17.816667
1           8.400000
2           8.966667
3          10.033333
4          37.533333
             ...    
2463926     5.966667
2463927    10.650000
2463928    11.000000
2463929    12.050000
2463930    27.000000
Name: duration, Length: 2463931, dtype: float64

In [13]:
print(f'Answer: {df_jan.duration.std():.2f}')

Answer: 46.45


# Question 3

In [23]:
duration_filter = df_jan.duration.between(1, 60)
df_jan_filtered = df_jan[duration_filter]

In [24]:
print(f'Answer: {len(df_jan_filtered) / len(df_jan) * 100:.0f}%')

Answer: 98%


# Question 4

In [25]:
from sklearn.feature_extraction import DictVectorizer

In [36]:
location_cols = ['PULocationID', 'DOLocationID']
df_jan_dict = df_jan_filtered[location_cols].astype(str).to_dict(orient='records')
df_jan_dict[0]

{'PULocationID': '142', 'DOLocationID': '236'}

In [37]:
dv = DictVectorizer()
df_jan_vect = dv.fit_transform(df_jan_dict)

In [40]:
print(f'Answer: {df_jan_vect.shape[1]}')

Answer: 515


# Question 5

In [41]:
X_train = df_jan_vect
y_train = df_jan_filtered.duration
X_train.shape, y_train.shape

((2421440, 515), (2421440,))

In [45]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [43]:
lr = LinearRegression()

In [44]:
lr.fit(X_train, y_train)

LinearRegression()

In [47]:
train_rmse = mean_squared_error(lr.predict(X_train), y_train, squared=False)
print(f'Answer: {train_rmse:.2f}')

Answer: 6.99


# Question 6

In [50]:
def prepare_data(filename: str):
    df = pd.read_parquet(filename)
    df['duration'] = (df.tpep_dropoff_datetime - df.tpep_pickup_datetime).dt.total_seconds() / 60
    df = df[df.duration.between(1, 60)]
    return (df[['PULocationID', 'DOLocationID']].astype(str).to_dict(orient='records'),
            df['duration'])

In [51]:
X_val, y_val = prepare_data('yellow_tripdata_2022-02.parquet')
X_val = dv.transform(X_val)
X_val.shape, y_val.shape

((2918187, 515), (2918187,))

In [52]:
val_rmse = mean_squared_error(lr.predict(X_val), y_val, squared=False)
print(f'Answer: {val_rmse:.2f}')

Answer: 7.79
